# Приложение для чтения книг - проект на языке SQL

# Задачи

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
- Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками;
- Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

# Загрузка библиотек и данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Таблица `books`

In [2]:
query = '''
SELECT *
FROM books
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблица `authors`

In [3]:
query = '''
SELECT *
FROM authors
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица `publishers`

In [4]:
query = '''
SELECT *
FROM publishers
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Таблица `ratings`

In [5]:
query = '''
SELECT *
FROM ratings
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица `reviews`

In [6]:
query = '''
SELECT *
FROM reviews
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# Задачи

## Посчитаем, сколько книг вышло после 1 января 2000 года

In [7]:
query = '''
SELECT COUNT(book_id) AS count_books
FROM books
WHERE publication_date > '2000-01-01'
'''
pd.io.sql.read_sql(query, con = engine)

,count_books
0,819


После 1 января 2000г. вышло 819 книг

## Для каждой книги посчитаем количество обзоров и среднюю оценку

In [8]:
query = '''
SELECT books.book_id,
       books.title, 
       COUNT(DISTINCT reviews.review_id) AS count_reviews, 
       AVG(ratings.rating) AS avg_rating
FROM books
LEFT JOIN reviews ON books.book_id = reviews.book_id
LEFT JOIN ratings ON books.book_id = ratings.book_id
            
GROUP BY books.book_id
ORDER BY avg_rating DESC

'''
pd.io.sql.read_sql(query, con = engine)


,book_id,title,count_reviews,avg_rating
0,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
1,901,The Walking Dead Book One (The Walking Dead #...,2,5.00
2,390,Light in August,2,5.00
3,972,Wherever You Go There You Are: Mindfulness Me...,2,5.00
4,136,Captivating: Unveiling the Mystery of a Woman'...,2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3,2.25
996,316,His Excellency: George Washington,2,2.00
997,202,Drowning Ruth,3,2.00
998,371,Junky,2,2.00


6 книг не имеет обзоров, хотя у книги с самым маленьким рейтингом - 1.5 есть 2 обзора, 43 книги имеет средний рейтинг 5

## Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [9]:
query = '''
SELECT publishers.publisher, 
       COUNT(books.book_id) AS count_books
FROM publishers
JOIN books ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher
ORDER BY COUNT(books.book_id) DESC
LIMIT 1;
'''
pd.io.sql.read_sql(query, con = engine)

,publisher,count_books
0,Penguin Books,42


Издательство Penguin Books выпустило наибольше число книг в количестве 42 книги

## Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками;

In [10]:
query = '''
SELECT authors.author, 
       AVG(ratings.rating)
FROM books
LEFT JOIN authors ON books.author_id = authors.author_id
LEFT JOIN ratings ON books.book_id = ratings.book_id
WHERE books.book_id IN
     (SELECT books.book_id
      FROM books
      LEFT JOIN ratings ON books.book_id = ratings.book_id        
      GROUP BY books.book_id
      HAVING count(ratings.rating) > 49)

GROUP BY authors.author
ORDER BY AVG(ratings.rating) DESC
LIMIT 1

'''
pd.io.sql.read_sql(query, con = engine)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.287097


Это автор `J.K. Rowling/Mary GrandPré` со средней оценкой 4.3

## Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [17]:
query = '''
SELECT AVG(table_1.count)
FROM 
    (SELECT count(*)
     FROM reviews
     WHERE username IN
          (SELECT username
           FROM ratings
           GROUP BY username
           HAVING count(rating) > 50)
     GROUP BY username) AS table_1;

'''
pd.io.sql.read_sql(query, con = engine)

,avg
0,24.333333


Среднее количество пользовательских обзоров, которые поставили больше 50 оценок - 24